In [1]:
!git clone https://github.com/Tanveer2719/NIDS_Coursework.git
%cd /kaggle/working/NIDS_Coursework/My_Code
!ls /kaggle/input


Cloning into 'NIDS_Coursework'...
remote: Enumerating objects: 262, done.
remote: Counting objects: 100% (262/262), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 262 (delta 126), reused 221 (delta 85), pack-reused 0 (from 0)
Receiving objects: 100% (262/262), 1.13 MiB | 9.71 MiB/s, done.
Resolving deltas: 100% (126/126), done.
/kaggle/working/NIDS_Coursework/My_Code
nslkdd


In [2]:
import os
import random
import numpy as np
import tensorflow as tf

def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.keras.utils.set_random_seed(seed)
    tf.config.experimental.enable_op_determinism()

set_seed(42)

2025-08-27 14:27:27.911429: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756304848.245348      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756304848.342728      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import pandas as pd
import numpy as np

url_train = "/kaggle/input/nslkdd/KDDTrain+.txt"
url_test = "/kaggle/input/nslkdd/KDDTest+.txt"
columns = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes',
    'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins',
    'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root',
    'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds',
    'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate',
    'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
    'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
    'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate', 'attack', 'level'
]

train_df = pd.read_csv(url_train, names=columns)
test_df = pd.read_csv(url_test, names=columns)


# Drop the "type" column (not needed)
train_df.drop(columns=['level'], inplace=True)
test_df.drop(columns=['level'], inplace=True)

# Strip whitespace from class labels
for df in [train_df, test_df]:
    df['attack'] = df['attack'].str.strip()



# Combine train and test
full_df = pd.concat([train_df, test_df], ignore_index=True)

full_df['label'] = full_df['attack'].apply(lambda x: 0 if x == 'normal' else 1)
full_df = full_df.drop(columns=['attack'])

print(f'full_df.shape : {full_df.shape}')




# --------------------------- perform preprocessing--------------

from preprocess import Preprocess

# separate the features and labels so that the labesl are not encoded
labels = full_df['label']
features = full_df.drop(columns=['label'])

pp = Preprocess()
processed_features = pp.fit_transform_df_auto(df = features,n_categorical_levels=32, expected_categorical_format='onehot')

print(processed_features.shape)

# concat the features and the labels
processed_full_df = pd.concat([processed_features, labels], axis=1)

# Filter out categorical one-hot encoded columns
numerical_columns = [col for col in processed_full_df 
                     if not (col.startswith("proto_") or 
                             col.startswith("service_") or 
                             col.startswith("state_") or
                             col == 'label')]

target = processed_full_df['label']
features = processed_full_df.drop(columns=['label'])


full_df.shape : (148517, 42)
Encoding the 3 levels for protocol_type
Encoding the 32 levels for service
Encoding the 11 levels for flag
(148517, 84)


In [4]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

X = features.astype("float32").values  # (257673, 95) as float32
y = target.astype("int32").values      # (257673,)


# reshape for (samples, timesteps, features=1)
X = np.expand_dims(X, axis=-1)  # (257673, 95, 1)

# Split train, val, test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42, stratify=y_temp)

print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"y_val: {y_val.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")



input_shape = (X_train.shape[1], X_train.shape[2])


X_train: (118813, 84, 1)
y_train: (118813,)
X_val: (23763, 84, 1)
y_val: (23763,)
X_test: (5941, 84, 1)
y_test: (5941,)


In [5]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

def evaluate_metrics(y_true, y_pred):
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        numerator = tp * tn - fp * fn
        denominator = ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) ** 0.5
        mcc = numerator / (denominator + 1e-10)
        specificity = tn / (tn + fp + 1e-10)
        balanced_acc = (recall + specificity) / 2
    else:
        mcc = 0
        balanced_acc = 0
        specificity = 0

    return {
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'mcc': mcc,
        'specificity': specificity,
        'balanced_accuracy': balanced_acc
    }

## Model Building

In [6]:
from tensorflow.keras import layers, models

# Bidirectional LSTM model
model = models.Sequential([
    layers.Bidirectional(layers.LSTM(64, return_sequences=False), input_shape=(95,1)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history_bi_lstm = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=256,
    validation_data=(X_val, y_val),
    verbose=1
)

# Predictions
y_pred_bi_lstm = (model.predict(X_test) > 0.5).astype("int32")

# Evaluate
bi_lstm_results = evaluate_metrics(y_true=y_test, y_pred=y_pred_bi_lstm)
print(f"Bidirectional LSTM results: {bi_lstm_results}")


I0000 00:00:1756304868.037291      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1756304868.038109      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20


2025-08-27 14:27:50.070178: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}
I0000 00:00:1756304873.805881     112 cuda_dnn.cc:529] Loaded cuDNN version 90300


464/465 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8321 - loss: 0.4216

2025-08-27 14:27:59.973515: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


465/465 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.8322 - loss: 0.4212 - val_accuracy: 0.9054 - val_loss: 0.2422
Epoch 2/20
465/465 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9175 - loss: 0.2149 - val_accuracy: 0.9325 - val_loss: 0.1798
Epoch 3/20
465/465 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9355 - loss: 0.1769 - val_accuracy: 0.9380 - val_loss: 0.1589
Epoch 4/20
465/465 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9384 - loss: 0.1569 - val_accuracy: 0.9360 - val_loss: 0.1572
Epoch 5/20
465/465 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9484 - loss: 0.1292 - val_accuracy: 0.9475 - val_loss: 0.1268
Epoch 6/20
465/465 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9557 - loss: 0.1146 - val_accuracy: 0.9639 - val_loss: 0.1051
Epoch 7/20
465/465 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9611 - loss: 0.1021 - val_accuracy: 0.9643 - val_loss: 0.1011
Epoch 8/20
465/465 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9649 - loss: 0.0944 - val_accuracy: 0.96

2025-08-27 14:29:45.196083: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Bidirectional LSTM results: {'f1': 0.9833596076370642, 'precision': 0.9849122807017544, 'recall': 0.9818118223154949, 'mcc': 0.9679745377873209, 'specificity': 0.9860480207657045, 'balanced_accuracy': 0.9839299215405997}


## Attack codes

In [7]:
import tensorflow as tf

@tf.function
def fgsm_attack_tf(model, inputs, labels, epsilon=0.05, clip_min=-1.5, clip_max=1.5):
    inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
    labels = tf.reshape(tf.convert_to_tensor(labels, dtype=tf.float32), (-1, 1))

    with tf.GradientTape() as tape:
        tape.watch(inputs)
        predictions = model(inputs, training=False)
        loss = tf.keras.losses.binary_crossentropy(labels, predictions)
        loss = tf.reduce_mean(loss)

    gradients = tape.gradient(loss, inputs)
    signed_grad = tf.sign(gradients)
    x_adv = inputs + epsilon * signed_grad
    x_adv = tf.clip_by_value(x_adv, clip_min, clip_max)
    return x_adv


@tf.function
def pgd_attack_tf(model, inputs, labels, epsilon=0.05, alpha=0.01, num_iter=10, clip_min=-1.5, clip_max=1.5):
    inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
    labels = tf.reshape(tf.convert_to_tensor(labels, dtype=tf.float32), (-1, 1))

    x_adv = tf.identity(inputs)

    for _ in tf.range(num_iter):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            predictions = model(x_adv, training=False)
            loss = tf.keras.losses.binary_crossentropy(labels, predictions)
            loss = tf.reduce_mean(loss)

        gradients = tape.gradient(loss, x_adv)
        x_adv = x_adv + alpha * tf.sign(gradients)
        x_adv = tf.clip_by_value(x_adv, inputs - epsilon, inputs + epsilon)
        x_adv = tf.clip_by_value(x_adv, clip_min, clip_max)

    return x_adv


@tf.function
def mi_fgsm_attack(model, inputs, labels, epsilon=0.05, alpha=0.01, steps=40, decay=1.0, clip_min=-1.5, clip_max=1.5):
    inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
    labels = tf.reshape(tf.convert_to_tensor(labels, dtype=tf.float32), (-1, 1))

    x_adv = tf.identity(inputs)
    g = tf.zeros_like(inputs)

    for _ in tf.range(steps):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            logits = model(x_adv, training=False)
            loss = tf.keras.losses.binary_crossentropy(labels, logits)
            loss = tf.reduce_mean(loss)

        grad = tape.gradient(loss, x_adv)
        # Normalize gradient
        grad_norm = tf.reduce_mean(tf.abs(grad), axis=list(range(1, len(grad.shape))), keepdims=True)
        grad = grad / (grad_norm + 1e-8)

        g = decay * g + grad
        x_adv = x_adv + alpha * tf.sign(g)
        x_adv = tf.clip_by_value(x_adv, inputs - epsilon, inputs + epsilon)
        x_adv = tf.clip_by_value(x_adv, clip_min, clip_max)

    return x_adv


In [8]:
def evaluate_model(model, X, y,epsilon_list=[0.01, 0.03, 0.05, 0.07]):
    import tensorflow as tf
    import numpy as np

    X_clean = X
    x_clean_tf = tf.convert_to_tensor(X_clean, dtype=tf.float32)
    y_tf = tf.convert_to_tensor(y, dtype=tf.float32)

    # ---------------- Clean Evaluation ----------------
    print("✅ Clean Evaluation:")
    y_pred_clean = (model.predict(x_clean_tf) > 0.5).astype(int).flatten()
    clean_metrics = evaluate_metrics(y, y_pred_clean)
    print(f"  F1: {clean_metrics['f1']:.4f}, Precision: {clean_metrics['precision']:.4f}, Recall: {clean_metrics['recall']:.4f}, MCC: {clean_metrics.get('mcc', 'N/A')}, Specificity: {clean_metrics.get('specificity', 'N/A')}, Balanced Acc: {clean_metrics.get('balanced_accuracy', 'N/A')}")

    # ---------------- FGSM Evaluation ----------------
    print("\nFGSM Attack Evaluation")
    fgsm_results = []
    for eps in epsilon_list:
        x_adv = fgsm_attack_tf(model, x_clean_tf, y_tf, epsilon=eps )
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        y_pred_adv = (model.predict(x_adv_tf) > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred_adv)
        fgsm_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f"  FGSM ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

    # ---------------- PGD Evaluation ----------------
    print("\nPGD Attack Evaluation:")
    pgd_results = []
    for eps in epsilon_list:
        alpha = eps * 0.2
        x_adv = pgd_attack_tf(model, x_clean_tf, y_tf, epsilon=eps, alpha=alpha, num_iter=10)
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        y_pred_adv = (model.predict(x_adv_tf) > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred_adv)
        pgd_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f"  PGD ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

    # ---------------- MI-FGSM Evaluation ----------------
    print("\nMI-FGSM Attack Evaluation:")
    mifgsm_results = []
    for eps in epsilon_list:
        alpha = eps * 0.2
        steps = 10   
        decay = 1.0   

        x_adv = mi_fgsm_attack(model, x_clean_tf, y_tf, epsilon=eps, alpha=alpha, steps=steps, decay=decay)
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        y_pred_adv = (model.predict(x_adv_tf) > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred_adv)
        mifgsm_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f" MI-FGSM ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

    return fgsm_results, pgd_results, mifgsm_results
        


In [9]:
print("CNN Results: ")
epsilons = [.01,.03,.05,.07]
fgsm_results, pgd_results, mifgsm_results = evaluate_model(
    model=model,
    X=X_test,
    y=y_test,
    epsilon_list=epsilons,
)



CNN Results: 
✅ Clean Evaluation:
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  F1: 0.9834, Precision: 0.9849, Recall: 0.9818, MCC: 0.9679745377873209, Specificity: 0.9860480207657045, Balanced Acc: 0.9839299215405997

FGSM Attack Evaluation
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  FGSM ε=0.01 → F1: 0.8948, Precision: 0.8850, Recall: 0.9049, MCC: 0.7953825767349914, Specificity: 0.890979883192703, Balanced Acc: 0.8979208614984151
 46/186 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

2025-08-27 14:29:50.725661: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
  FGSM ε=0.03 → F1: 0.6634, Precision: 0.5479, Recall: 0.8409, MCC: 0.2241431039620976, Specificity: 0.35626216742373923, Balanced Acc: 0.5985578063421599
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  FGSM ε=0.05 → F1: 0.6489, Precision: 0.5819, Recall: 0.7335, MCC: 0.2500860476810722, Specificity: 0.5110317975340521, Balanced Acc: 0.6222525199632485
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
  FGSM ε=0.07 → F1: 0.6204, Precision: 0.5745, Recall: 0.6744, MCC: 0.21267598788308995, Specificity: 0.5366645035690936, Balanced Acc: 0.6055130842434485

PGD Attack Evaluation:
 48/186 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

2025-08-27 14:29:57.644154: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
  PGD ε=0.01 → F1: 0.8436, Precision: 0.8284, Recall: 0.8594, MCC: 0.693750452514361, Specificity: 0.834847501622296, Balanced Acc: 0.8471194486075804
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
  PGD ε=0.03 → F1: 0.5635, Precision: 0.4722, Recall: 0.6985, MCC: -0.028362158253871866, Specificity: 0.2757949383517107, Balanced Acc: 0.4871454579831306
 49/186 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

...

186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
  PGD ε=0.05 → F1: 0.4043, Precision: 0.3519, Recall: 0.4750, MCC: -0.35239033650710483, Specificity: 0.18851395197922816, Balanced Acc: 0.33175260383151683
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
  PGD ε=0.07 → F1: 0.2134, Precision: 0.2017, Recall: 0.2267, MCC: -0.6071990403182418, Specificity: 0.1677482154445111, Balanced Acc: 0.1972004456026333

MI-FGSM Attack Evaluation:
 49/186 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

2025-08-27 14:30:11.583776: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
 MI-FGSM ε=0.01 → F1: 0.8526, Precision: 0.8416, Recall: 0.8639, MCC: 0.7126107699108889, Specificity: 0.849123945489914, Balanced Acc: 0.8565311927519524
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
 MI-FGSM ε=0.03 → F1: 0.5533, Precision: 0.4560, Recall: 0.7034, MCC: -0.08567722277150489, Specificity: 0.22160934458143344, Balanced Acc: 0.4625010696324446
 46/186 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

2025-08-27 14:30:18.552315: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
 MI-FGSM ε=0.05 → F1: 0.4109, Precision: 0.3473, Recall: 0.5030, MCC: -0.40669590685762, Specificity: 0.12297209604152748, Balanced Acc: 0.31297258177382425
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
 MI-FGSM ε=0.07 → F1: 0.1944, Precision: 0.1752, Recall: 0.2183, MCC: -0.7491274581734098, Specificity: 0.047047371836468295, Balanced Acc: 0.13265275202526458


## Adversarial Training

In [10]:
import tensorflow as tf
from tensorflow.keras import Model

class PGDAdversarialTrainerCNN(tf.keras.Model):
    def __init__(self, base_model, epsilon_range=(0.01, 0.1), alpha=0.01, num_iter=7, clip_min=-1.5, clip_max=1.5):
        super().__init__()
        self.base_model = base_model
        self.epsilon_range = epsilon_range
        self.alpha = alpha
        self.num_iter = num_iter
        self.clip_min = clip_min
        self.clip_max = clip_max
        self.loss_fn = tf.keras.losses.BinaryCrossentropy()
        self.metric = tf.keras.metrics.BinaryAccuracy()

    def compile(self, optimizer, **kwargs):
        super().compile(**kwargs)
        self.optimizer = optimizer

    def pgd_attack_batch(self, x, y):
        x_adv = tf.identity(x)
        y = tf.expand_dims(tf.cast(y, tf.float32), axis=-1)
        epsilon = tf.random.uniform([], *self.epsilon_range)

        for _ in range(self.num_iter):
            with tf.GradientTape() as tape:
                tape.watch(x_adv)
                preds = self.base_model(x_adv, training=True)
                loss = self.loss_fn(y, preds)

            grad = tape.gradient(loss, x_adv)
            x_adv = x_adv + self.alpha * tf.sign(grad)
            # Project back to epsilon ball around original input
            x_adv = tf.clip_by_value(x_adv, x - epsilon, x + epsilon)
            x_adv = tf.clip_by_value(x_adv, self.clip_min, self.clip_max)

        return x_adv

    def train_step(self, data):
        x, y = data
        x_adv = self.pgd_attack_batch(x, y)

        # Combine clean and adversarial samples
        x_combined = tf.concat([x, x_adv], axis=0)
        y_combined = tf.concat([y, y], axis=0)
        y_combined = tf.expand_dims(tf.cast(y_combined, tf.float32), axis=-1)

        with tf.GradientTape() as tape:
            preds = self.base_model(x_combined, training=True)
            loss = self.loss_fn(y_combined, preds)

        grads = tape.gradient(loss, self.base_model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.base_model.trainable_variables))
        self.metric.update_state(y_combined, preds)

        return {"loss": loss, "accuracy": self.metric.result()}

    def test_step(self, data):
        x, y = data
        y = tf.expand_dims(tf.cast(y, tf.float32), axis=-1)
        preds = self.base_model(x, training=False)
        loss = self.loss_fn(y, preds)
        self.metric.update_state(y, preds)
        return {"loss": loss, "accuracy": self.metric.result()}

    def call(self, inputs):
        return self.base_model(inputs)


In [11]:
# Wrap your existing CNN

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',   # monitor validation loss
    patience=3,           # stop if no improvement for 3 epochs
    restore_best_weights=True
)


adv_model = PGDAdversarialTrainerCNN(
    base_model=model,           
    epsilon_range=(0.01, 0.05),
    alpha=0.01,
    num_iter=7,
    clip_min=-1.5,
    clip_max=1.5
)

adv_model.compile(optimizer=tf.keras.optimizers.Adam())

# Train with adversarial training
adv_model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=256,
    epochs=20,
    callbacks=[early_stop] 

)

print("Advesarial training Results: ")
epsilons = [.01,.03,.05,.07]
fgsm_results, pgd_results, mifgsm_results = evaluate_model(
    model=adv_model,
    X=X_test,
    y=y_test,
    epsilon_list=epsilons,
)


Epoch 1/20
464/465 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.8360 - loss: 0.3123

2025-08-27 14:31:10.320764: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


465/465 ━━━━━━━━━━━━━━━━━━━━ 48s 78ms/step - accuracy: 0.8361 - loss: 0.3116 - val_accuracy: 0.9241 - val_loss: 0.1983
Epoch 2/20
465/465 ━━━━━━━━━━━━━━━━━━━━ 35s 76ms/step - accuracy: 0.9116 - loss: 0.1972 - val_accuracy: 0.9660 - val_loss: 0.0751
Epoch 3/20
465/465 ━━━━━━━━━━━━━━━━━━━━ 35s 76ms/step - accuracy: 0.9451 - loss: 0.1386 - val_accuracy: 0.9672 - val_loss: 0.0550
Epoch 4/20
465/465 ━━━━━━━━━━━━━━━━━━━━ 35s 76ms/step - accuracy: 0.9556 - loss: 0.1191 - val_accuracy: 0.9699 - val_loss: 0.0489
Epoch 5/20
465/465 ━━━━━━━━━━━━━━━━━━━━ 36s 76ms/step - accuracy: 0.9603 - loss: 0.1049 - val_accuracy: 0.9758 - val_loss: 0.0368
Epoch 6/20
465/465 ━━━━━━━━━━━━━━━━━━━━ 35s 76ms/step - accuracy: 0.9649 - loss: 0.0936 - val_accuracy: 0.9803 - val_loss: 0.0339
Epoch 7/20
465/465 ━━━━━━━━━━━━━━━━━━━━ 35s 75ms/step - accuracy: 0.9675 - loss: 0.0854 - val_accuracy: 0.9824 - val_loss: 0.0314
Epoch 8/20
465/465 ━━━━━━━━━━━━━━━━━━━━ 35s 76ms/step - accuracy: 0.9704 - loss: 0.0808 - val_accurac

2025-08-27 14:42:23.346059: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
  F1: 0.9890, Precision: 0.9885, Recall: 0.9895, MCC: 0.9787628323905042, Specificity: 0.9892926670992541, Balanced Acc: 0.9893997438329429

FGSM Attack Evaluation
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
  FGSM ε=0.01 → F1: 0.9872, Precision: 0.9867, Recall: 0.9878, MCC: 0.9753917722331293, Specificity: 0.9876703439324793, Balanced Acc: 0.9877141506301081
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
  FGSM ε=0.03 → F1: 0.9839, Precision: 0.9832, Recall: 0.9846, MCC: 0.9689875357469794, Specificity: 0.9844256975989297, Balanced Acc: 0.9845178505483281
 44/186 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

...

186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
  PGD ε=0.05 → F1: 0.9747, Precision: 0.9735, Recall: 0.9759, MCC: 0.9511247744708912, Specificity: 0.9753406878649911, Balanced Acc: 0.975603187584122
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  PGD ε=0.07 → F1: 0.9354, Precision: 0.9362, Recall: 0.9346, MCC: 0.8755965945856612, Specificity: 0.940947436729366, Balanced Acc: 0.9377699757973518

MI-FGSM Attack Evaluation:
 44/186 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

2025-08-27 14:42:47.839929: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
 MI-FGSM ε=0.01 → F1: 0.9872, Precision: 0.9867, Recall: 0.9878, MCC: 0.9753917722331293, Specificity: 0.9876703439324793, Balanced Acc: 0.9877141506301081
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
 MI-FGSM ε=0.03 → F1: 0.9825, Precision: 0.9818, Recall: 0.9832, MCC: 0.966290720672714, Specificity: 0.9831278390655099, Balanced Acc: 0.9831693759860602
 48/186 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

2025-08-27 14:42:54.830641: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
 MI-FGSM ε=0.05 → F1: 0.9757, Precision: 0.9749, Recall: 0.9766, MCC: 0.9531448274541383, Specificity: 0.9766385463984109, Balanced Acc: 0.9766018894986108
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
 MI-FGSM ε=0.07 → F1: 0.9469, Precision: 0.9462, Recall: 0.9475, MCC: 0.8975219362789499, Specificity: 0.9500324464633046, Balanced Acc: 0.9487832746482315


In [12]:
# generate adversarial values for gan training

# experimental PGD code (GPU-optimized, batched)
def pgd_attack_tf_fast(model, x, y, epsilon=0.05, alpha=0.01, num_iter=10, clip_min=-1.5, clip_max=1.5):
    x = tf.convert_to_tensor(x, dtype=tf.float32)
    y = tf.reshape(tf.cast(y, tf.float32), (-1, 1))
    x_orig = tf.identity(x)
    x_adv = tf.identity(x)

    for _ in range(num_iter):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            logits = model(x_adv, training=False)
            loss = tf.keras.losses.binary_crossentropy(y, logits)
            loss = tf.reduce_mean(loss)

        grad = tape.gradient(loss, x_adv)
        signed_grad = tf.sign(grad)
        x_adv = x_adv + alpha * signed_grad
        x_adv = tf.clip_by_value(x_adv, x_orig - epsilon, x_orig + epsilon)
        x_adv = tf.clip_by_value(x_adv, clip_min, clip_max)

    return x_adv

def batch_pgd_attack(model, X, y, attack_fn, batch_size=1024, **kwargs):
    adv_list = []
    X = tf.convert_to_tensor(X, dtype=tf.float32)
    y = tf.convert_to_tensor(y, dtype=tf.float32)
    for i in range(0, len(X), batch_size):
        X_batch = X[i:i+batch_size]
        y_batch = y[i:i+batch_size]
        x_adv_batch = attack_fn(model, X_batch, y_batch, **kwargs)
        adv_list.append(x_adv_batch)
    return tf.concat(adv_list, axis=0)

# Generate adversarial examples in batches to leverage GPU
X_adv_train = batch_pgd_attack(model, X_train, y_train, pgd_attack_tf_fast, epsilon=0.05, batch_size=1024)
print(f"X_adv_train shape: {X_adv_train.shape}")

X_adv_val = batch_pgd_attack(model, X_val, y_val, pgd_attack_tf_fast, epsilon=0.05, batch_size=1024)
print(f"X_adv_val shape: {X_adv_val.shape}")


X_adv_train shape: (118813, 84, 1)
X_adv_val shape: (23763, 84, 1)


In [13]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import EarlyStopping

# ------------------------------
# Generator CNN (1D output)
# ------------------------------
class GeneratorCNN(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = layers.Conv1D(64, 3, padding='same', activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv1D(128, 3, padding='same', activation='relu')
        self.bn2 = layers.BatchNormalization()
        self.conv3 = layers.Conv1D(64, 3, padding='same', activation='relu')
        self.bn3 = layers.BatchNormalization()
        # final output: 1 channel (same as clean input)
        self.output_layer = layers.Conv1D(1, 3, padding='same', activation='linear')

    def call(self, x, training=False):
        x = self.conv1(x)
        x = self.bn1(x, training=training)
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.conv3(x)
        x = self.bn3(x, training=training)
        return self.output_layer(x)


# ------------------------------
# Discriminator CNN (1D input)
# ------------------------------
class DiscriminatorCNN(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = layers.Conv1D(64, 3, padding='same', activation='relu')
        self.global_pool = layers.GlobalAveragePooling1D()
        self.dense1 = layers.Dense(64, activation='relu')
        self.output_layer = layers.Dense(1, activation='sigmoid')

    def call(self, x, training=False):
        x = self.conv1(x)
        x = self.global_pool(x)
        x = self.dense1(x)
        return self.output_layer(x)


# ------------------------------
# GAN Purifier CNN
# ------------------------------
class GANPurifierCNN(Model):
    def __init__(self, generator, discriminator, lambda_adv=0.5):
        super().__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.lambda_adv = lambda_adv
        self.bce = tf.keras.losses.BinaryCrossentropy()
        self.l1 = tf.keras.losses.MeanAbsoluteError()
        self.d_optimizer = tf.keras.optimizers.Adam(1e-4)
        self.g_optimizer = tf.keras.optimizers.Adam(1e-4)

    def compile(self, **kwargs):
        super().compile(**kwargs)

    def train_step(self, data):
        x_clean, x_adv = data
        with tf.GradientTape(persistent=True) as tape:
            x_fake = self.generator(x_adv, training=True)
            real_pred = self.discriminator(x_clean, training=True)
            fake_pred = self.discriminator(x_fake, training=True)

            # Discriminator loss
            d_loss = self.bce(tf.ones_like(real_pred), real_pred) + \
                     self.bce(tf.zeros_like(fake_pred), fake_pred)

            # Generator loss (reconstruction + adversarial)
            g_loss = self.l1(x_clean, x_fake) + \
                     self.lambda_adv * self.bce(tf.ones_like(fake_pred), fake_pred)

            # Cosine similarity
            x_clean_flat = tf.reshape(x_clean, (tf.shape(x_clean)[0], -1))
            x_fake_flat = tf.reshape(x_fake, (tf.shape(x_fake)[0], -1))
            cos_sim = -tf.reduce_mean(
                tf.keras.losses.cosine_similarity(x_clean_flat, x_fake_flat)
            )

        # Compute gradients
        d_grads = tape.gradient(d_loss, self.discriminator.trainable_variables)
        g_grads = tape.gradient(g_loss, self.generator.trainable_variables)

        # Apply gradients
        self.d_optimizer.apply_gradients(zip(d_grads, self.discriminator.trainable_variables))
        self.g_optimizer.apply_gradients(zip(g_grads, self.generator.trainable_variables))

        return {"d_loss": d_loss, "g_loss": g_loss,
                "reconstruction_loss": self.l1(x_clean, x_fake),
                "cosine_similarity": cos_sim}





In [14]:
# ------------------------------
# Instantiate models
# ------------------------------
generator = GeneratorCNN()
discriminator = DiscriminatorCNN()

# Build models with dummy input (1 channel)
dummy_input = tf.zeros((1, 95, 1))
generator(dummy_input)
discriminator(dummy_input)

gan_cnn = GANPurifierCNN(generator, discriminator, lambda_adv=0.5)
gan_cnn.compile()

# ------------------------------
# Prepare dataset
# ------------------------------
batch_size = 512
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, X_adv_train))
train_dataset = train_dataset.shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

early_stop = EarlyStopping(
    monitor="cosine_similarity",
    mode="max",              # higher cosine similarity is better
    patience=10,
    min_delta=1e-4,
    restore_best_weights=True,
    verbose=1
)

# ------------------------------
# Train
# ------------------------------
gan_cnn.fit(train_dataset, epochs=100, callbacks=[early_stop])


Epoch 1/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 13s 26ms/step - cosine_similarity: 0.8517 - d_loss: 1.3775 - g_loss: 0.4628 - reconstruction_loss: 0.1136
Epoch 2/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - cosine_similarity: 0.9717 - d_loss: 1.3800 - g_loss: 0.3909 - reconstruction_loss: 0.0424
Epoch 3/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - cosine_similarity: 0.9814 - d_loss: 1.3791 - g_loss: 0.3819 - reconstruction_loss: 0.0333
Epoch 4/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - cosine_similarity: 0.9855 - d_loss: 1.3782 - g_loss: 0.3776 - reconstruction_loss: 0.0288
Epoch 5/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - cosine_similarity: 0.9877 - d_loss: 1.3772 - g_loss: 0.3756 - reconstruction_loss: 0.0266
Epoch 6/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - cosine_similarity: 0.9890 - d_loss: 1.3759 - g_loss: 0.3745 - reconstruction_loss: 0.0252
Epoch 7/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - cosine_similarity: 0.9897 - d_loss: 1.3727 - g_loss: 0.3748 - recon

In [15]:
# Generate purified outputs
x_purified_val = generator(X_adv_val, training=False)

# Cast to float32 for evaluation
x_purified_val = tf.cast(x_purified_val, tf.float32)
X_val_cast = tf.cast(X_val, tf.float32)

# L1 (Mean Absolute Error)
l1_loss = tf.reduce_mean(tf.abs(X_val_cast - x_purified_val))

# L2 (Mean Squared Error)
l2_loss = tf.reduce_mean(tf.square(X_val_cast - x_purified_val))

# Normalize along sequence dimension
X_val_norm = tf.nn.l2_normalize(tf.squeeze(X_val_cast, axis=-1), axis=1)           
x_purified_norm = tf.nn.l2_normalize(tf.squeeze(x_purified_val, axis=-1), axis=1)  

# Cosine similarity
cos_sim = tf.reduce_mean(tf.reduce_sum(X_val_norm * x_purified_norm, axis=1))

print(f"🧪 Purifier Evaluation Metrics:")
print(f"✅ Cosine Similarity: {cos_sim:.6f}")
print(f"   ➤ L1 Loss (MAE): {l1_loss:.6f}")
print(f"   ➤ L2 Loss (MSE): {l2_loss:.6f}")


🧪 Purifier Evaluation Metrics:
✅ Cosine Similarity: 0.991643
   ➤ L1 Loss (MAE): 0.021814
   ➤ L2 Loss (MSE): 0.001656


## GAN+Adv Training evaluation

In [16]:
def evaluate_model(model, generator, adv_model, X, y, epsilon_list=[0.01, 0.03, 0.05, 0.07]):
    import tensorflow as tf
    import numpy as np

    X_clean = X
    x_clean_tf = tf.convert_to_tensor(X_clean, dtype=tf.float32)
    y_tf = tf.convert_to_tensor(y, dtype=tf.float32)

    # ----------- Clean Data Evaluation (no adversarial) -----------
    print("✅ Clean Evaluation (GAN purified):")
    x_purified_clean = generator(x_clean_tf, training=False)
    y_pred_clean_prob = adv_model.predict(x_purified_clean)
    y_pred_clean = (y_pred_clean_prob > 0.5).astype(int).flatten()
    clean_metrics = evaluate_metrics(y, y_pred_clean)
    print(f"Clean → F1: {clean_metrics['f1']:.4f}, Precision: {clean_metrics['precision']:.4f}, Recall: {clean_metrics['recall']:.4f}, MCC: {clean_metrics.get('mcc', 'N/A')}, Specificity: {clean_metrics.get('specificity', 'N/A')}, Balanced Acc: {clean_metrics.get('balanced_accuracy', 'N/A')}")

    # ----------- FGSM Attack Evaluation -----------
    print("\nFGSM Attack Evaluation with GAN Purification")
    fgsm_results = []
    for eps in epsilon_list:
        x_adv = fgsm_attack_tf(model, x_clean_tf, y_tf, epsilon=eps )
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        x_purified = generator(x_adv_tf, training=False)
        y_pred_prob = adv_model.predict(x_purified)
        y_pred = (y_pred_prob > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred)
        fgsm_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f"  FGSM ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

    # ----------- PGD Attack Evaluation -----------
    print("\nPGD Attack Evaluation with GAN Purification:")
    pgd_results = []
    for eps in epsilon_list:
        alpha = eps * 0.2
        x_adv = pgd_attack_tf(model, x_clean_tf, y_tf, epsilon=eps, alpha=alpha, num_iter=10)
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        x_purified = generator(x_adv_tf, training=False)
        y_pred_prob = adv_model.predict(x_purified )
        y_pred = (y_pred_prob > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred)
        pgd_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f"  PGD ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

      # ----------- MI-FGSM Attack Evaluation -----------
    print("\nMI-FGSM Attack Evaluation with GAN Purification:")
    mifgsm_results = []
    for eps in epsilon_list:
        alpha = eps * 0.2
        steps = 10
        decay = 1.0

        x_adv = mi_fgsm_attack(model, x_clean_tf, y_tf, epsilon=eps, alpha=alpha, steps=steps, decay=decay)
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        x_purified = generator(x_adv_tf, training=False)
        y_pred_prob = adv_model.predict(x_purified )
        y_pred = (y_pred_prob > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred)
        mifgsm_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f"  MI-FGSM ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

    return fgsm_results, pgd_results, mifgsm_results


In [17]:
fgsm_full, pgd_full, mi_full = evaluate_model(model= model,generator=generator,adv_model= model, X = X_test, y = y_test)


✅ Clean Evaluation (GAN purified):
 46/186 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

2025-08-27 14:47:17.257732: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Clean → F1: 0.9869, Precision: 0.9881, Recall: 0.9857, MCC: 0.9747165766461235, Specificity: 0.9889682024658991, Balanced Acc: 0.9873137619534812

FGSM Attack Evaluation with GAN Purification
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
  FGSM ε=0.01 → F1: 0.9848, Precision: 0.9846, Recall: 0.9850, MCC: 0.9706711045559225, Specificity: 0.9857235561323495, Balanced Acc: 0.9853416661389275
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  FGSM ε=0.03 → F1: 0.9792, Precision: 0.9797, Recall: 0.9787, MCC: 0.9598818497283655, Specificity: 0.9811810512653802, Balanced Acc: 0.9799224598754324
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  FGSM ε=0.05 → F1: 0.9724, Precision: 0.9727, Recall: 0.9720, MCC: 0.9467341219330121, Specificity: 0.9746917585982812, Balanced Acc: 0.9733549733879828
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  FGSM ε=0.07 → F1: 0.9605, Precision: 0.9647, Recall: 0.9563, MCC: 0.9241571456650681, Specificity: 0.9675535366644722, Balanced Acc: 0.9619

2025-08-27 14:47:23.707196: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
  PGD ε=0.01 → F1: 0.9844, Precision: 0.9839, Recall: 0.9850, MCC: 0.9699980759813296, Specificity: 0.9850746268656396, Balanced Acc: 0.9850172015055725
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  PGD ε=0.03 → F1: 0.9757, Precision: 0.9749, Recall: 0.9766, MCC: 0.9531448274541383, Specificity: 0.9766385463984109, Balanced Acc: 0.9766018894986108
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
  PGD ε=0.05 → F1: 0.9696, Precision: 0.9686, Recall: 0.9706, MCC: 0.9413477352119551, Specificity: 0.9707981829980217, Balanced Acc: 0.9707086402922952
 46/186 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

2025-08-27 14:47:31.049410: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  PGD ε=0.07 → F1: 0.9613, Precision: 0.9631, Recall: 0.9594, MCC: 0.9254927738468393, Specificity: 0.9659312134976974, Balanced Acc: 0.9626787931776699

MI-FGSM Attack Evaluation with GAN Purification:
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  MI-FGSM ε=0.01 → F1: 0.9848, Precision: 0.9846, Recall: 0.9850, MCC: 0.9706711045559225, Specificity: 0.9857235561323495, Balanced Acc: 0.9853416661389275
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  MI-FGSM ε=0.03 → F1: 0.9778, Precision: 0.9776, Recall: 0.9780, MCC: 0.9571865288009188, Specificity: 0.9792342634652504, Balanced Acc: 0.9785992933275884
 44/186 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

2025-08-27 14:47:38.477554: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  MI-FGSM ε=0.05 → F1: 0.9722, Precision: 0.9707, Recall: 0.9738, MCC: 0.9464085950920117, Specificity: 0.9727449707981515, Balanced Acc: 0.9732560111073654
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  MI-FGSM ε=0.07 → F1: 0.9570, Precision: 0.9605, Recall: 0.9535, MCC: 0.9174069815261596, Specificity: 0.9636599610642127, Balanced Acc: 0.9585700994548065
